In [927]:
import pandas as pd
import numpy as np 
import csv
import re
from difflib import SequenceMatcher

### **Problem 01:**

In [928]:
file_input = '../Data/temp_data.csv'
file_output = '../Data/clean_data.csv'

In [929]:
# Hàm sửa lỗi dư hoặc thiếu dấu ngoặc kép trong file CSV
def fix_csv_quotes(file_input, file_output):
    # Mở file đầu vào để đọc và file đầu ra để ghi
    with open(file_input, 'r', encoding = 'utf-8') as infile, open(file_output, 'w', newline = '', encoding = 'utf-8') as outfile:
        # Đọc tất cả các dòng từ file đầu vào
        reader = infile.readlines()
        # Tạo writer để ghi vào file đầu ra
        writer = csv.writer(outfile)

        # Lặp qua từng dòng trong file đầu vào
        for line in reader:
            # Kiểm tra số lượng dấu ngoặc kép trong dòng
            num_quotes = line.count('"')
            # Nếu số lượng dấu ngoặc kép là số lẻ, thêm một dấu ngoặc kép ở cuối dòng
            if num_quotes % 2 != 0:
                line = line.strip() + '"\n'
            # Ghi lại dòng đã sửa vào file đầu ra
            writer.writerow(next(csv.reader([line], skipinitialspace = True)))

In [930]:
fix_csv_quotes(file_input, file_output)

### **Problem 02:**

In [931]:
column_index = 8  # Chỉ số của cột cần làm sạch (bắt đầu từ 0), cột 8 là cột Description

def clean_string(string):

    """
    Hàm này sẽ làm sạch chuỗi bằng cách loại bỏ các biểu tượng và ký tự không cần thiết.
    (nhưng giữ lại các kí tự chữ (hoa và thường), số, dấu chấm, dấu phẩy, dấu | )
    
    string: Chuỗi cần làm sạch.
    cleanString: Chuỗi đã được làm sạch.

    """

    # Hàm isinstance check xem data input của mình có phải là string không 
    if isinstance(string, str):
        cleanString = re.sub(r'[^\w\s.,|]', '', string) # Xoá tất cả các kí tự ngoại trừ chữ (thường, hoa), số, dấu chấm, dấu phẩy, dấu |
        return cleanString
    else:
        return string

In [932]:
def clean_file_csv(file_input, file_output, column_index):

    """
    Hàm này sẽ làm sạch một cột cụ thể trong file CSV, do có những cột khác (VD cột Postdate có dấu : ), 
    Vì vậy nếu apply full file csv nó sẽ fix hết cả những cột không cần thiết -> chỉ áp dụng chính xác cột cần xử lí thôi.

    input_csv (str): Đường dẫn tới file CSV đầu vào (temp_data).
    output_csv (str): Đường dẫn tới file CSV đầu ra (clean_data).
    column_index (int): Chỉ số của cột cần làm sạch.
    """

    # Đọc file CSV vào DataFrame
    df = pd.read_csv(file_input, on_bad_lines = 'skip')

    # Kiểm tra xem chỉ số cột có hợp lệ không
    if column_index < len(df.columns):
        # Áp dụng hàm làm sạch cho cột cụ thể
        col = df.columns[column_index]
        df[col] = df[col].apply(clean_string)
    else:
        print(f"Invalid column index !")

    # Lưu lại DataFrame đã được làm sạch thành file CSV mới
    df.to_csv(file_output, index = False)



In [933]:
clean_file_csv(file_input, file_output, column_index)

### **Problem 03:**

In [934]:
# Đọc lại file csv, xoá các dòng bị lỗi 
# (mình nhận thấy số lượng dòng bị lỗi không nhiều ~ 10 dòng nên mình sẽ skip luôn vì tụi mình có nhiều data mà =))))
df = pd.read_csv('../Data/clean_data.csv', on_bad_lines = 'skip')

In [935]:
# Loại bỏ cột No và cột Title không cần thiết 
df = df.drop(columns = df.columns[[0, 2]])

# Thay thế "Thoả thuận" bằng NaN cho dễ xử lí về sau (tạm thời)
df['Price'] = df['Price'].replace('TT', np.nan)

# Thiết lập ngưỡng cho số lượng giá trị NaN tối đa cho phép trong một hàng
max_nan = 3  

# Xóa các hàng có số lượng giá trị NaN lớn hơn ngưỡng
df = df.dropna(thresh = len(df.columns) - max_nan)  # thresh: chỉ định số lượng giá trị NOT NULL cần thiết để giữ lại hàng

In [936]:
# Chuyển đổi các cột 'Price' và 'Area' sang kiểu số (float)
df['Price'] = df['Price'].astype(float)
df['Area'] = df['Area'].astype(float)

In [937]:
# Hàm xử lý cột Price
def process_price(df):
    def convert_price(price):
        if pd.isnull(price):
            return price
        price = float(price)
        if price > 1e9:
            return price / 1e9  # Nếu giá trị lớn hơn 1 tỷ, chuyển sang đơn vị tỷ
        elif price > 1e6:
            return price / 1e6  # Nếu giá trị lớn hơn 1 triệu, chuyển sang đơn vị triệu
        elif price > 1e3:
            return price / 1e3  # Nếu giá trị lớn hơn 1 nghìn, chuyển sang đơn vị nghìn
        return price  # Trả về giá trị gốc nếu nhỏ hơn 1 nghìn
    
    df['Price'] = df['Price'].apply(convert_price)  # Áp dụng hàm convert_price cho cột 'Price'
    return df

df = process_price(df)

# Hàm tính giá trên một mét vuông
def price_per_sqm(df):
    df['Price_per_sqm'] = df.apply(lambda row: row['Price'] / row['Area'] if pd.notnull(row['Price']) and pd.notnull(row['Area']) else np.nan, axis=1)
    # Tạo cột 'Price_per_sqm' bằng cách lấy 'Price' chia cho 'Area' nếu cả hai đều không bị null, ngược lại trả về NaN
    return df

# Hàm điền giá trị thiếu
def fill_miss_vals(df):
    # Tính giá trị trung bình của 'Price', 'Area', và 'Price_per_sqm' theo 'District'
    district_means = df.groupby('District').mean(numeric_only=True)
    
    # Điền giá trị thiếu cho 'Price' bằng giá trị trung bình của 'District'
    for idx, row in df.iterrows():
        if pd.isnull(row['Price']):
            df.at[idx, 'Price'] = district_means.loc[row['District'], 'Price']
    
    # Tính lại giá trên một mét vuông
    df = price_per_sqm(df)
    
    # Điền giá trị thiếu cho 'Area' và 'Price' dựa trên 'Price_per_sqm'
    for idx, row in df.iterrows():
        if pd.isnull(row['Area']) and pd.notnull(row['Price']):
            df.at[idx, 'Area'] = row['Price'] / row['Price_per_sqm']  # Tính diện tích dựa trên giá trị 'Price' và 'Price_per_sqm'
        elif pd.isnull(row['Price']) and pd.notnull(row['Area']):
            df.at[idx, 'Price'] = row['Area'] * row['Price_per_sqm']  # Tính giá trị 'Price' dựa trên 'Area' và 'Price_per_sqm'
        elif pd.isnull(row['Price']) and pd.isnull(row['Area']):
            df.at[idx, 'Price'] = district_means.loc[row['District'], 'Price']  # Điền 'Price' bằng giá trị trung bình của quận
            df.at[idx, 'Area'] = district_means.loc[row['District'], 'Area']  # Điền 'Area' bằng giá trị trung bình của quận
    
    # Điền giá trị thiếu cho 'Price_per_sqm' nếu còn thiếu
    for idx, row in df.iterrows():
        if pd.isnull(row['Price_per_sqm']) and pd.notnull(row['Price']) and pd.notnull(row['Area']):
            df.at[idx, 'Price_per_sqm'] = row['Price'] / row['Area']  # Tính lại 'Price_per_sqm' nếu thiếu và các giá trị khác không bị null
    
    # # Xử lý các giá trị 'Area' vẫn còn thiếu
    for idx, row in df.iterrows():
        if pd.isnull(row['Area']):  
            df.at[idx, 'Area'] = district_means.loc[row['District'], 'Area']

    # Làm tròn các giá trị vừa điền vào
    df['Price'] = df['Price'].round(2)  # Làm tròn giá trị 'Price' đến 2 chữ số thập phân
    df['Area'] = df['Area'].round(2)  # Làm tròn giá trị 'Area' đến 2 chữ số thập phân
    df['Price_per_sqm'] = df['Price_per_sqm'].round(2)  # Làm tròn giá trị 'Price_per_sqm' đến 2 chữ số thập phân

    return df

# Thêm cột giá trên 1 mét vuông
df = price_per_sqm(df)

# Điền giá trị thiếu cho 'Price' và 'Area'
df = fill_miss_vals(df)


In [938]:
# Kiểm tra giá trị null trong cột 'Area' và 'Price'
null_area = df['Area'].isnull().sum()
print(null_area)
null_price = df['Price'].isnull().sum()
print(null_price)

0
0


In [939]:
# Xử lí cột Postdate (chỉ giữ lại Ngày xoá Giờ)
# Chuyển đổi cột 'Postdate' sang kiểu datetime
df['Postdate'] = pd.to_datetime(df['Postdate'])

# Tạo một cột mới chỉ chứa ngày
df['Date'] = df['Postdate'].dt.date

# Tìm vị trí (index) của cột 'Postdate'
col_index = df.columns.get_loc('Postdate')

# Xóa cột 'Postdate' cũ 
df = df.drop(columns = ['Postdate'])

# Chèn cột 'Date' mới vào đúng vị trí cũ của cột 'Postdate'
df.insert(col_index, 'Postdate', df.pop('Date'))

In [940]:
# Load lại những thay đổi vào file clean_data.csv
df.to_csv(file_output, index = False)

In [941]:
df.head(20)

,Price,Area,Bedrooms,WCs,District,Postdate,Description,Price_per_sqm
3,13.5,44.00,NaN,NaN,quan-1,2023-11-24,"Chủ kẹt cần bán gấp nhà 2 MT Hoàng Sa, Phường ...",0.31
7,8.7,45.00,3.0,3.0,quan-1,2023-11-27,"Vị trí khu vực ngay đường Trần Hưng Đạo, đối ...",0.19
8,8.7,45.00,3.0,3.0,quan-1,2023-11-27,"Vị trí khu vực ngay đường Trần Hưng Đạo, đối ...",0.19
9,21.5,64.00,NaN,NaN,quan-1,2023-11-27,"Vị trí trung tâm Quận 1, khách thượng lưu và ...",0.34
10,21.5,64.00,NaN,NaN,quan-1,2023-11-27,"Vị trí trung tâm Quận 1, khách thượng lưu và ...",0.34
11,9.3,40.00,4.0,4.0,quan-1,2023-11-29,"Kết cấu 1 trệt, 2 lầu, sân thượng, 3 PN, 4WC....",0.23
12,130.0,455.00,NaN,NaN,quan-1,2023-10-25,Diện tích 13 x 35m. | Hiện trạng Nhà cấp 4. |...,0.29
13,60.0,114.40,NaN,NaN,quan-1,2023-10-25,Diện tích 5.2 x 22m. | Kết cấu 1 tầng. | Tuyế...,0.52
14,42.0,145.00,NaN,NaN,quan-1,2023-10-25,Diện tích 5 x 29m. | Kết cấu Hầm 7 tầng. | T...,0.29
15,115.0,324.00,NaN,NaN,quan-1,2023-10-25,Vị trí đẹp ngay Ngã Sáu Phù Đổng siêu đắc địa...,0.35


In [942]:
def remove_unusual_line_terminators(file_input, file_output):
    with open(file_input, 'r', encoding='utf-8') as infile:
        content = infile.read()

    # Loại bỏ các ký tự kết thúc dòng không bình thường (LS và PS)
    cleaned_content = content.replace('\u2028', '\n').replace('\u2029', '\n')

    with open(file_output, 'w', encoding='utf-8', newline='') as outfile:
        outfile.write(cleaned_content)

In [943]:
def remove_duplicates(df):
    # Đảm bảo rằng các giá trị trong 'Postdate' và 'Description' là chuỗi
    df['Postdate'] = df['Postdate'].astype(str)
    df['Description'] = df['Description'].astype(str)
    
    # Bước 1: Loại bỏ các dòng trùng lặp mà tất cả các cột đều giống nhau
    df = df.drop_duplicates()
    
    # Bước 2: Nhóm các dòng theo các cột 'Price', 'Area', 'Bedrooms', 'WCs', 'District'
    grouped = df.groupby(['Price', 'Area', 'Bedrooms', 'WCs', 'District'], dropna=False)
    
    # Danh sách các chỉ số của các dòng cần giữ lại
    indices_to_keep = set(df.index)

    def similar(a, b):
        return SequenceMatcher(None, a, b).ratio()
    
    # Lặp qua từng nhóm để kiểm tra sự tương đồng
    for _, group in grouped:
        group_indices = list(group.index)
        for i in range(len(group_indices)):
            for j in range(i + 1, len(group_indices)):
                idx1, idx2 = group_indices[i], group_indices[j]
                row1, row2 = df.loc[idx1], df.loc[idx2]
                # Kiểm tra sự tương đồng của 'Description' và ngày 'Postdate'
                postdate_similarity = similar(row1['Postdate'], row2['Postdate'])
                description_similarity = similar(row1['Description'], row2['Description'])
                if postdate_similarity > 0.75 and description_similarity > 0.5:
                    indices_to_keep.discard(idx2)
    
    # Chuyển đổi tập hợp các chỉ số cần giữ lại thành danh sách
    indices_to_keep = list(indices_to_keep)
    
    # Lọc lại DataFrame với các chỉ số cần giữ lại
    df_cleaned = df.loc[indices_to_keep].reset_index(drop=True)
    
    # Trả về DataFrame đã loại bỏ các dòng trùng lặp
    return df_cleaned

In [944]:
df.head(20)

,Price,Area,Bedrooms,WCs,District,Postdate,Description,Price_per_sqm
3,13.5,44.00,NaN,NaN,quan-1,2023-11-24,"Chủ kẹt cần bán gấp nhà 2 MT Hoàng Sa, Phường ...",0.31
7,8.7,45.00,3.0,3.0,quan-1,2023-11-27,"Vị trí khu vực ngay đường Trần Hưng Đạo, đối ...",0.19
8,8.7,45.00,3.0,3.0,quan-1,2023-11-27,"Vị trí khu vực ngay đường Trần Hưng Đạo, đối ...",0.19
9,21.5,64.00,NaN,NaN,quan-1,2023-11-27,"Vị trí trung tâm Quận 1, khách thượng lưu và ...",0.34
10,21.5,64.00,NaN,NaN,quan-1,2023-11-27,"Vị trí trung tâm Quận 1, khách thượng lưu và ...",0.34
11,9.3,40.00,4.0,4.0,quan-1,2023-11-29,"Kết cấu 1 trệt, 2 lầu, sân thượng, 3 PN, 4WC....",0.23
12,130.0,455.00,NaN,NaN,quan-1,2023-10-25,Diện tích 13 x 35m. | Hiện trạng Nhà cấp 4. |...,0.29
13,60.0,114.40,NaN,NaN,quan-1,2023-10-25,Diện tích 5.2 x 22m. | Kết cấu 1 tầng. | Tuyế...,0.52
14,42.0,145.00,NaN,NaN,quan-1,2023-10-25,Diện tích 5 x 29m. | Kết cấu Hầm 7 tầng. | T...,0.29
15,115.0,324.00,NaN,NaN,quan-1,2023-10-25,Vị trí đẹp ngay Ngã Sáu Phù Đổng siêu đắc địa...,0.35


In [945]:
# Áp dụng hàm remove_duplicates lên DataFrame đã được làm sạch
df = remove_duplicates(df)

In [946]:
# DataFrame sau khi thêm cột "No" và loại bỏ duplicate 
df.head(20)

,Price,Area,Bedrooms,WCs,District,Postdate,Description,Price_per_sqm
0,13.5,44.00,NaN,NaN,quan-1,2023-11-24,"Chủ kẹt cần bán gấp nhà 2 MT Hoàng Sa, Phường ...",0.31
1,8.7,45.00,3.0,3.0,quan-1,2023-11-27,"Vị trí khu vực ngay đường Trần Hưng Đạo, đối ...",0.19
2,21.5,64.00,NaN,NaN,quan-1,2023-11-27,"Vị trí trung tâm Quận 1, khách thượng lưu và ...",0.34
3,9.3,40.00,4.0,4.0,quan-1,2023-11-29,"Kết cấu 1 trệt, 2 lầu, sân thượng, 3 PN, 4WC....",0.23
4,130.0,455.00,NaN,NaN,quan-1,2023-10-25,Diện tích 13 x 35m. | Hiện trạng Nhà cấp 4. |...,0.29
5,60.0,114.40,NaN,NaN,quan-1,2023-10-25,Diện tích 5.2 x 22m. | Kết cấu 1 tầng. | Tuyế...,0.52
6,42.0,145.00,NaN,NaN,quan-1,2023-10-25,Diện tích 5 x 29m. | Kết cấu Hầm 7 tầng. | T...,0.29
7,115.0,324.00,NaN,NaN,quan-1,2023-10-25,Vị trí đẹp ngay Ngã Sáu Phù Đổng siêu đắc địa...,0.35
8,65.0,171.00,NaN,NaN,quan-1,2023-10-25,Vị trí đẹp vô cùng đắc địa và sang trọng. | D...,0.38
9,65.0,288.00,NaN,NaN,quan-1,2023-10-25,Vị trí cực đẹp góc 3 mặt tiền thông thoáng. |...,0.23


In [947]:
# Xem kiểu dữ liệu của cột "Postdate"
df['Postdate'].dtype

dtype('O')

In [948]:
# Chuyển kiểu dữ liệu của cột "Postdate" sang dạng datetime và check lại kiểu dữ liệu 1 lần nữa
df['Postdate'] = pd.to_datetime(df['Postdate'])
df['Postdate'].dtype

dtype('<M8[ns]')

In [949]:
# Loại bỏ các data có ngày đăng tin là 2022 và lưu lại vào file csv
df = df[df['Postdate'].dt.year != 2022]

In [950]:
# Tạo lại cột No
df.insert(0, 'No', range(1, len(df) + 1))

# Lưu lại DataFrame vào file CSV
df.to_csv(file_output, index = False)